<a href="https://colab.research.google.com/github/anny11020/Colab/blob/main/%E8%A8%93%E7%B7%B4%E6%89%8B%E5%8B%A212345_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 訓練手勢 - CNN

In [ ]:
import numpy as np
import os
import cv2
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

# 設個picture大小及data路徑
pic_size = 128
image_path = '/content/drive/My Drive/Colab Notebooks/Gesture/dataset/traindata/'

# 印出dataset中各類有幾張image
for image_count in os.listdir(image_path):
    print(str(len(os.listdir(image_path + image_count))) + " " + image_count + " images")

# 記錄總共有幾張image
file_count = 0
for floderName in os.listdir(image_path):
    for filename in os.listdir(image_path + floderName):
        file_count +=1
print('all_image_file: ',file_count)

# 建立空的np_array (待會填label用)
label_default = np.zeros(shape=[file_count])
img_default = np.zeros(shape=[file_count,pic_size,pic_size])
file_count = 0

# 給各個floder中的image上label
for floderName in os.listdir(image_path):
    for filename in os.listdir(image_path + floderName):

        temp = cv2.imread(image_path + floderName + "/" + filename,0)
        temp = cv2.resize(temp, (pic_size,pic_size))
        img_default[file_count] = temp

        if floderName == '1':
            label_default[file_count] = 0
        elif floderName == '2':
            label_default[file_count] = 1
        elif floderName == '3':
            label_default[file_count] = 2
        elif floderName == '4':
            label_default[file_count] = 3
        elif floderName == '5':
            label_default[file_count] = 4

        file_count +=1

# reshape成丟進model input的dimension
img_default = img_default.reshape(file_count,pic_size,pic_size,1)
img_default.shape

label_onehot=np_utils.to_categorical(label_default) # 做onehot encoding
print('label_onehot[0]:{},label_dim:{},shape:{}'.format(label_onehot[0],label_onehot.ndim,label_onehot.shape)) # Label(Encoding結果 , 維度, shape)
img_default = img_default / 255.0 # 做 normalization


random_seed  = 3 # 隨機分割
x_train, x_test, y_train, y_test = train_test_split(img_default, label_onehot, test_size = 0.2, random_state=random_seed) # 切分訓練及測試集
print('x_train.shape:{}\n,y_train.shape:{}\nx_test.shape:{}\ny_test.shape:{}'.format(x_train.shape, y_train.shape, x_test.shape, y_test.shape)) #(train_img, train_label, test_img, test_label)


5 4 images
5 3 images
5 2 images
5 1 images
5 5 images
all_image_file:  25
label_onehot[0]:[0. 0. 0. 1. 0.],label_dim:2,shape:(25, 5)
x_train.shape:(20, 128, 128, 1)
,y_train.shape:(20, 5)
x_test.shape:(5, 128, 128, 1)
y_test.shape:(5, 5)


# 儲存 model

In [ ]:
# ... preprocessing ...

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

classes = 5 # 有五種分類

model = Sequential([
    Conv2D(64, 3, activation='relu', input_shape=(pic_size,pic_size,1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(classes, activation='softmax')
])

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50)

# 將訓練好的model儲存成json及h5檔
import json
model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/Gesture/model_trained.json", "w") as json_file:
    json.dump(model_json, json_file)
model.save("/content/drive/My Drive/Colab Notebooks/Gesture/model_trained.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3686528   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

# h5 轉成 pb
https://stackoverflow.com/questions/60005661/tensorflow-2-0-convert-keras-model-to-pb-file

In [ ]:
import tensorflow as tf

pre_model = tf.keras.models.load_model("final_model.h5")
pre_model.save("saved_model")

# pb 轉成 byte
https://github.com/llSourcell/Unity_ML_Agents/blob/master/docs/Using-TensorFlow-Sharp-in-Unity-(Experimental).md

# 擷取影像 - OpenCV

In [ ]:
import cv2
cap = cv2.VideoCapture(0)

while True:
    _, FrameImage = cap.read() # 讀取鏡頭畫面
    FrameImage = cv2.flip(FrameImage, 1) # 圖像水平翻轉
    cv2.imshow("Webcam", FrameImage) # 顯示鏡頭畫面
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == ord('q'): # 觸及小寫q，關閉webcam
      break

cap.release()
cv2.destroyAllWindows()

# 蒐集 Data

In [ ]:
import cv2

# ... get webcam and ROI_preprocessed ...

while True:
  cv2.imwrite('./test/handdata'+str(i)+'.jpg',output2)
  i += 1
  cv2.waitKey(100)

# 畫面前處理

In [ ]:
import cv2
CLIP_X1,CLIP_Y1,CLIP_X2,CLIP_Y2 = 160,140,400,360 # ROI's size
i = 0 # 用來記錄之後要新增的image
wzs = 158 # 調整二值化的閥值變數
image_q = cv2.THRESH_BINARY # 調整二值化的模式

# ... Get webcam ...

cv2.rectangle(FrameImage, (CLIP_X1, CLIP_Y1), (CLIP_X2, CLIP_Y2), (0,255,0) ,1) # 框出ROI位置
ROI = FrameImage[CLIP_Y1:CLIP_Y2, CLIP_X1:CLIP_X2] # ROI的大小
ROI = cv2.resize(ROI, (128, 128))  # ROI RESIZE (配合訓練大小)
ROI = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY) # ROI 轉灰階
SHOWROI = cv2.resize(ROI, (256, 256)) # ROI resize
# Threshold Binary：即二值化，將大於閾值的灰度值設為最大灰度值，小於閾值的值設為0。
_, output = cv2.threshold(ROI, wzs, 255, image_q) # Black Background is better for prediction
_, output2 = cv2.threshold(SHOWROI, wzs, 255, image_q) # 用來顯示
cv2.imshow("ROI", output2)

# 預測及辨識

In [ ]:
import json
import pygame
import sys, os
import operator
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

# 使用pygame來建一個預測視窗
pygame.init()
screen = pygame.display.set_mode((400,400),pygame.RESIZABLE)

# ... Get webcam ... ( in while True )

# loading model
with open('model_in_json_CNN_SW_IOU.json','r') as f:
    model_json = json.load(f)
loaded_model = model_from_json(model_json)
loaded_model.load_weights('model_Binary_CNN_SW_IOU.h5')
# ... ROI_preprocessed ...
result = loaded_model.predict(output2.reshape(1,128, 128, 1)) # 若是訓練彩色，則須把1改成3個channel (1,128,128,3) 配合model輸入的dimension
predict =   { '1':    result[0][0], # 看訓練幾類，給予多少的result
              '2':    result[0][1],
              '3':    result[0][2],
              '4':    result[0][3],
              '5':    result[0][4],
            }
# print(predict)  # 有需要可以print出來看一下各類predict的分數
predict = sorted(predict.items(), key=operator.itemgetter(1), reverse=True) # 分數較高者會sort至第一位

# 這邊是取對應預測的image，沒有則是顯示nosign
if(predict[0][1] == 1.0):
  predict_img  = pygame.image.load(os.getcwd() + '/dataset/images/' + predict[0][0] + '.jpg')
else:
  predict_img  = pygame.image.load(os.getcwd() + '/dataset/images/nosign.png')
predict_img = pygame.transform.scale(predict_img, (400, 400))
screen.blit(predict_img, (0,0))
pygame.display.flip()

pygame.quit()

# 新增 Command

In [ ]:
import cv2

# ... get webcam and ROI_preprocessed ...

while True:

  # 在ROI上觸及英數字，來反饋效果

  interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == ord('l'): # lower wzs quality (小寫'l'改threshold的wzs，將標準調低)
      wzs = wzs - 5
    elif interrupt & 0xFF == ord('u'): # upper wzs quality (小寫'u'改threshold的wzs，將標準調高)
      wzs = wzs + 5
    elif interrupt & 0xFF == ord ('s'): # save dataset (小寫's'儲存當下frame)
      cv2.imwrite('handdata'+str(random.randint(1,9999))+'.jpg',output2)
    elif interrupt & 0xFF == ord ('c'): # change THRESH_BINARY TO THRESH_BINARY_INV (小寫'c'改treshold的image_q，背景黑白對調)
      if image_q == cv2.THRESH_BINARY_INV:
        image_q = cv2.THRESH_BINARY
      else:
        image_q = cv2.THRESH_BINARY_INV
    if interrupt & 0xFF == ord('q'): # esc key
        break